# Group Name & numer: DLNK, #26
## Group members:
Natiq Khan (nak9135@nyu.edu)\
David Lopez (dld388@nyu.edu)

Our project is divided into 4 notebooks, each serving its own unique function:
1. Notebook-1: Data retrieval and loading
2. **→Notebook-2: Data cleaning and standardizing**
3. Notebook-3: Exploratory analysis and visualizations
4. Notebook-4: Machine Learning and predictions

We made each notebook in keeping with the principles of **modularization and testing**. As well as **abstraction**, which made it easier to share our work between members seamlessly. 

Data retrieval was done in Notebook-1. **Before you proceed with Notebook-2, please ensure you have the `pbp_csv` folder with 10 CSV files in it as well as the `moneyline.csv` file.**

# 2.1 Configuration

In [1]:
# For file handling
import os
# For pathname handling
import glob
# The usual
import numpy as np
import pandas as pd

# Relative path from current working directory
DATA_DIR = "./pbp_csv"

# 2.2 Selecting columns to use:

**Challenge**: with our current dataset, we had 1.5 million+ rows and 372 columns across the 10 datasets combined. This made the data very slow to load and manipulate. However, based on our NFL-domain knowledge we knew that only a few of these columns are relevant for our analysis.

Thus, we chose to filter out the most useful columns only:

In [2]:
# Based on NFL-specific knowledge, we were able to narrow down to:
USECOLS = [
    "game_id",
    "season",
    "season_type",
    "week",
    "game_date",
    "home_team",
    "away_team",

    # score columns
    "total_home_score",
    "total_away_score",
    "yards_gained",

    # play context
    "posteam",
    "defteam",
    "pass",
    "rush",
    "touchdown",
    "interception",
    "fumble_lost",
    "epa",
]

# 2.3 Data aggregation:
Time to load data into df and aggregate it so that each row is 1 game instead of 1 play. 

The following code:
- Loads all CSVs from directory
- Extracts a minimal set of play-level columns
- Aggregates each game to one row with ~25 meaningful features
- Produces `games_df` for downstream modeling

In [3]:
# Functions for loading our data

def list_pbp_files(data_dir: str) -> list[str]:
    """
    Return a sorted list of play-by-play CSV file paths in `data_dir`.
    """
    files = glob.glob(os.path.join(data_dir, "*.csv"))
    files.sort()
    return files


def load_pbp(path: str, usecols=None) -> pd.DataFrame:
    """
    Load a single play-by-play CSV into a DataFrame.

    Parameters
    ----------
    path : str
        Path to the CSV file.
    usecols : list[str] or None
        Column subset to read. If None, all columns are loaded.

    Returns
    -------
    pandas.DataFrame
    """
    # This handles the error where you are not in the correct directory
    # Or have not downloaded the needed CSV files yet
    if not os.path.exists(path):
        raise FileNotFoundError(f"PBP file not found: {path}")
    return pd.read_csv(path, usecols=usecols)

Now that we have functions for loading the data, we next write a function to aggregate our data. We want 1 row per game, instead of 1 row per play. 

In [4]:
# The "explosive" play threshold is subjective
# But based on experience, we decided to set at 20
EXPLOSIVE_YARD_THRESHOLD = 20

# Aggregation functions:

# This function further calls another function called team_agg()
def aggregate_game(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate a play-level DataFrame into one row per game.

    Features produced (per game):
      - Metadata: season, season_type, week, date, home_team, away_team
      - Final scores and score differential
      - Home win flag
      - For each team (home/away):
          * total plays
          * pass plays
          * rush plays
          * total yards
          * total EPA
          * mean EPA
          * success rate (EPA > 0)
          * touchdowns
          * interceptions
          * fumbles lost
          * explosive plays (yards_gained >= EXPLOSIVE_YARD_THRESHOLD)
    """

    # Ensure key stat columns are numeric / 0-1 for safe aggregation
    for col in ["pass", "rush", "touchdown", "interception", "fumble_lost"]:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)

    for col in ["yards_gained", "epa"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Final score and basic game info
    score_agg = (
        df.groupby("game_id")
          .agg(
              season=("season", "first"),
              season_type=("season_type", "first"),
              week=("week", "first"),
              game_date=("game_date", "first"),
              home_team=("home_team", "first"),
              away_team=("away_team", "first"),
              final_home_score=("total_home_score", "max"),
              final_away_score=("total_away_score", "max"),
          )
    )

    score_agg["score_diff"] = (
        score_agg["final_home_score"] - score_agg["final_away_score"]
    )
    score_agg["home_win"] = (score_agg["score_diff"] > 0).astype(int)

    # Identify plays belonging to home vs away offense

    # Bring home/away team labels down to play level (for older pbp versions, this is usually already present)
    df = df.merge(
        score_agg[["home_team", "away_team"]],
        left_on="game_id",
        right_index=True,
        how="left",
        suffixes=("", "_g"),
    )

    df["is_home_play"] = df["posteam"] == df["home_team"]
    df["is_away_play"] = df["posteam"] == df["away_team"]

    home = df[df["is_home_play"]]
    away = df[df["is_away_play"]]

    # Per-team aggregations

    def team_agg(sub: pd.DataFrame, prefix: str) -> pd.DataFrame:
        """
        Aggregate per-team (home/away) play-level stats within each game.
        """
        return sub.groupby("game_id").agg(
            **{
                f"{prefix}_plays": ("epa", "count"),
                f"{prefix}_pass_plays": ("pass", "sum"),
                f"{prefix}_rush_plays": ("rush", "sum"),
                f"{prefix}_yards_total": ("yards_gained", "sum"),
                f"{prefix}_epa_total": ("epa", "sum"),
                f"{prefix}_epa_mean": ("epa", "mean"),
                f"{prefix}_success_rate": ("epa", lambda x: np.mean(x > 0)),
                f"{prefix}_touchdowns": ("touchdown", "sum"),
                f"{prefix}_interceptions": ("interception", "sum"),
                f"{prefix}_fumbles_lost": ("fumble_lost", "sum"),
                f"{prefix}_explosive_plays": ("yards_gained",
                                              lambda x: np.sum(x >= EXPLOSIVE_YARD_THRESHOLD)),
            }
        )

    home_stats = team_agg(home, "home")
    away_stats = team_agg(away, "away")

    # ---- Combine into final game-level table

    games = (
        score_agg
        .join(home_stats, how="left")
        .join(away_stats, how="left")
        .reset_index()
    )

    return games

Now that all functions are in place, we can apply them to our data:

In [5]:
# BUILD FINAL GAME-LEVEL DATAFRAME

all_games = []

for path in list_pbp_files(DATA_DIR):
    print(f"Loading {path}")
    df = load_pbp(path, usecols=USECOLS)
    games = aggregate_game(df)
    all_games.append(games)
    del df  # free memory

games_df = pd.concat(all_games, ignore_index=True)

Loading ./pbp_csv/play_by_play_2005.csv
Loading ./pbp_csv/play_by_play_2006.csv
Loading ./pbp_csv/play_by_play_2007.csv
Loading ./pbp_csv/play_by_play_2008.csv
Loading ./pbp_csv/play_by_play_2009.csv
Loading ./pbp_csv/play_by_play_2010.csv
Loading ./pbp_csv/play_by_play_2011.csv
Loading ./pbp_csv/play_by_play_2012.csv
Loading ./pbp_csv/play_by_play_2013.csv
Loading ./pbp_csv/play_by_play_2014.csv
Loading ./pbp_csv/play_by_play_2015.csv


Let's see how our data looks now:

In [6]:
print("\nFinal game-level shape:", games_df.shape)
pd.set_option('display.max_columns', None)
games_df.head()


Final game-level shape: (2937, 33)


,game_id,season,season_type,week,game_date,home_team,away_team,final_home_score,final_away_score,score_diff,home_win,home_plays,home_pass_plays,home_rush_plays,home_yards_total,home_epa_total,home_epa_mean,home_success_rate,home_touchdowns,home_interceptions,home_fumbles_lost,home_explosive_plays,away_plays,away_pass_plays,away_rush_plays,away_yards_total,away_epa_total,away_epa_mean,away_success_rate,away_touchdowns,away_interceptions,away_fumbles_lost,away_explosive_plays
0,2005_01_ARI_NYG,2005,REG,1,2005-09-11,NYG,ARI,42,19,23,1,73,28,24,275.0,6.759280,0.092593,0.472973,6,2,0,6,107,65,18,318.0,-12.868063,-0.120262,0.420561,2,2,0,2
1,2005_01_CHI_WAS,2005,REG,1,2005-09-11,WAS,CHI,9,7,2,1,82,31,37,323.0,-7.139239,-0.087064,0.397590,0,1,2,3,68,32,19,166.0,-8.244967,-0.121250,0.411765,1,1,1,2
2,2005_01_CIN_CLE,2005,REG,1,2005-09-11,CLE,CIN,13,27,-14,0,76,43,18,373.0,-9.822493,-0.129243,0.493506,1,2,1,3,87,39,33,420.0,8.030959,0.092310,0.620690,3,1,1,3
3,2005_01_DAL_SD,2005,REG,1,2005-09-11,LAC,DAL,24,28,-4,0,77,38,27,291.0,9.087176,0.118015,0.480519,3,2,0,2,85,32,35,301.0,16.470139,0.193766,0.465116,4,0,1,3
4,2005_01_DEN_MIA,2005,REG,1,2005-09-11,MIA,DEN,34,10,24,1,93,40,33,426.0,5.094963,0.054785,0.446809,3,1,1,3,89,55,18,312.0,-17.160922,-0.192819,0.426966,2,2,2,3


Now that we have 33 columns and about 3000 games worth of data, it is much more manageable.\
We are readdy to add the underdog labels now. 

# 2.4 Underdog labels
For this part of our analysis, we will load another dataset `moneyline.csv` which contains the Moneyline data for these games. We will use this data to label our `games_df` data with underdog-win-related information. 

The below code will: 
1. Load the moneyline dataset
2. Normallize team codes to account for team name changes
3. Determine Home and Away teams for each game
4. Identify "Underdog" team using the money spread
5. Label games for when underdog team won
6. Collapse the moneyline data to have game-wise rows
7. Merge the moneyline dataset with our `game_df`

In [7]:
# 1. LOAD MONEYLINE DATASET

money = pd.read_csv("./moneyline.csv")

# Columns present: Season, Team, Opp, Spread, Result, etc.

In [8]:
# 2. TEAM CODE NORMALIZATION (PFR → PBP CODES)
# PFR codes differ (e.g., GNB → GB, KAN → KC, SDG → SD/LAC, OAK → LV)
# This mapping handles all 32 teams historically.

pfr_to_pbp = {
    'GNB': 'GB',
    'KAN': 'KC',
    'NWE': 'NE',
    'NWE': 'NE',
    'SFO': 'SF',
    'NOR': 'NO',
    'SDG': 'SD',     # your PBP 2005–2015 sample uses SD (not LAC)
    'OAK': 'OAK',    # PBP files preserve historical name; use LV only in modern years
    'STL': 'STL',    # Rams were STL during 2005–2015
    # Identity mappings (3-letter PFR matches PBP)
    'BUF': 'BUF', 'MIA': 'MIA', 'NYJ': 'NYJ', 'NYG': 'NYG',
    'DAL': 'DAL', 'WAS': 'WAS', 'PHI': 'PHI',
    'CHI': 'CHI', 'DET': 'DET', 'MIN': 'MIN',
    'ATL': 'ATL', 'CAR': 'CAR', 'TAM': 'TB', 'TBB': 'TB',
    'ARI': 'ARI', 'SEA': 'SEA',
    'DEN': 'DEN',
    'PIT': 'PIT', 'BAL': 'BAL', 'CLE': 'CLE', 'CIN': 'CIN',
    'TEN': 'TEN', 'HOU': 'HOU', 'IND': 'IND', 'JAX': 'JAX',
}

# Extract raw opponent team code (strip leading "@")
money["opp_raw"] = money["Opp"].str.replace("@", "", regex=False)

# Normalize both teams
money["team_pbp"] = money["Team"].map(pfr_to_pbp)
money["opp_pbp"]  = money["opp_raw"].map(pfr_to_pbp)

In [9]:
# 3. DETERMINE HOME vs AWAY BASED ON "Opp"

money["is_away_team"] = money["Opp"].str.startswith("@")

money["home_team"] = np.where(money["is_away_team"], money["opp_pbp"], money["team_pbp"])
money["away_team"] = np.where(money["is_away_team"], money["team_pbp"], money["opp_pbp"])

In [10]:
# 4. IDENTIFY THE UNDERDOG USING SPREAD
# Spread > 0  → underdog
# Spread < 0  → favorite
# Spread = 0  → pick’em

money["is_underdog"] = money["Spread"] > 0

In [11]:
# 5. DID THE UNDERDOG WIN?
# Result field looks like "W, 27-6" → check first letter only.

money["team_won"] = money["Result"].str.startswith("W")

# Under-dog win at the team-row level
money["underdog_and_won"] = money["is_underdog"] & money["team_won"]

In [12]:
# 6. COLLAPSE TO GAME-LEVEL (one record per game)
# A game is uniquely defined by: Season, home_team, away_team

game_lines = (
    money.groupby(["Season", "home_team", "away_team"], as_index=False)
         .agg(
             underdog_win=("underdog_and_won", "any"),
             underdog_team=("team_pbp", lambda x: x.loc[money.loc[x.index, "is_underdog"]].iloc[0]
                                          if (money.loc[x.index, "is_underdog"]).any()
                                          else None),
             spread_home=("Spread", lambda x:
                          x.loc[money.loc[x.index, "team_pbp"] == money.loc[x.index, "home_team"]].iloc[0]
                          if (money.loc[x.index, "team_pbp"] == money.loc[x.index, "home_team"]).any()
                          else np.nan),
             spread_away=("Spread", lambda x:
                          x.loc[money.loc[x.index, "team_pbp"] == money.loc[x.index, "away_team"]].iloc[0]
                          if (money.loc[x.index, "team_pbp"] == money.loc[x.index, "away_team"]).any()
                          else np.nan),
         )
)

game_lines["underdog_win"] = game_lines["underdog_win"].astype(int)  # convert bool → 0/1

In [13]:
# 7. MERGE WITH YOUR GAME-LEVEL PBP DATAFRAME (games_df)

# Important: rename 'Season' to 'season' for merge
game_lines = game_lines.rename(columns={"Season": "season"})

games_df = games_df.merge(
    game_lines,
    how="left",
    on=["season", "home_team", "away_team"],
)

print("Merged shape:", games_df.shape)
games_df.head()

Merged shape: (2937, 37)


,game_id,season,season_type,week,game_date,home_team,away_team,final_home_score,final_away_score,score_diff,home_win,home_plays,home_pass_plays,home_rush_plays,home_yards_total,home_epa_total,home_epa_mean,home_success_rate,home_touchdowns,home_interceptions,home_fumbles_lost,home_explosive_plays,away_plays,away_pass_plays,away_rush_plays,away_yards_total,away_epa_total,away_epa_mean,away_success_rate,away_touchdowns,away_interceptions,away_fumbles_lost,away_explosive_plays,underdog_win,underdog_team,spread_home,spread_away
0,2005_01_ARI_NYG,2005,REG,1,2005-09-11,NYG,ARI,42,19,23,1,73,28,24,275.0,6.759280,0.092593,0.472973,6,2,0,6,107,65,18,318.0,-12.868063,-0.120262,0.420561,2,2,0,2,0.0,ARI,-1.5,1.5
1,2005_01_CHI_WAS,2005,REG,1,2005-09-11,WAS,CHI,9,7,2,1,82,31,37,323.0,-7.139239,-0.087064,0.397590,0,1,2,3,68,32,19,166.0,-8.244967,-0.121250,0.411765,1,1,1,2,0.0,CHI,-6.0,6.0
2,2005_01_CIN_CLE,2005,REG,1,2005-09-11,CLE,CIN,13,27,-14,0,76,43,18,373.0,-9.822493,-0.129243,0.493506,1,2,1,3,87,39,33,420.0,8.030959,0.092310,0.620690,3,1,1,3,0.0,CLE,3.5,-3.5
3,2005_01_DAL_SD,2005,REG,1,2005-09-11,LAC,DAL,24,28,-4,0,77,38,27,291.0,9.087176,0.118015,0.480519,3,2,0,2,85,32,35,301.0,16.470139,0.193766,0.465116,4,0,1,3,NaN,NaN,NaN,NaN
4,2005_01_DEN_MIA,2005,REG,1,2005-09-11,MIA,DEN,34,10,24,1,93,40,33,426.0,5.094963,0.054785,0.446809,3,1,1,3,89,55,18,312.0,-17.160922,-0.192819,0.426966,2,2,2,3,1.0,MIA,4.5,-4.5


Now that we have a game-wise dataset with underdog labels, we can move on to data cleaning. 

# 2.5 Data cleaning:
First, let's check if any rows are missing their target label (`underdog_win`):

In [14]:
missing = games_df['underdog_win'].isna().sum()
missing

592

In [15]:
missing_pct = games_df['underdog_win'].isna().mean() * 100
missing_pct

20.156622403813415

About 20% seem to be missing their moneyline data on BOTH sides. We have no choice but to drop those:

In [16]:
games_df = games_df.dropna(subset=["underdog_win"]).reset_index(drop=True)
games_df.shape

(2345, 37)

However, some rows seem to be missing ONLY ONE SIDE of the spread (either home or away) which results in the `underdog_team` label also being missing. However, the other side of the spread can be trivially computed given one side, and with that we can also compute the `underdog_team` label for those which are missing it:

In [17]:
import numpy as np

# ------------------------------------------------------------------
# 1. Fill missing spreads from the opposite side
#    Relationship: spread_home = -spread_away
# ------------------------------------------------------------------

# Fill spread_home when missing but spread_away is present
mask_home_na = games_df["spread_home"].isna() & games_df["spread_away"].notna()
games_df.loc[mask_home_na, "spread_home"] = -games_df.loc[mask_home_na, "spread_away"]

# Fill spread_away when missing but spread_home is present
mask_away_na = games_df["spread_away"].isna() & games_df["spread_home"].notna()
games_df.loc[mask_away_na, "spread_away"] = -games_df.loc[mask_away_na, "spread_home"]

# Optional: quick sanity check
print("Remaining NaNs in spreads:")
print(games_df[["spread_home", "spread_away"]].isna().sum())


# ------------------------------------------------------------------
# 2. Infer missing underdog_team from spreads
#    Convention: spread > 0 → that team is the underdog
# ------------------------------------------------------------------

mask_ud_na = games_df["underdog_team"].isna()

home_is_underdog = games_df["spread_home"] > 0
away_is_underdog = games_df["spread_away"] > 0

# If home spread > 0, home is underdog
games_df.loc[mask_ud_na & home_is_underdog, "underdog_team"] = \
    games_df.loc[mask_ud_na & home_is_underdog, "home_team"]

# If away spread > 0, away is underdog
games_df.loc[mask_ud_na & away_is_underdog, "underdog_team"] = \
    games_df.loc[mask_ud_na & away_is_underdog, "away_team"]

# Optional: remaining rows with no underdog_team (likely true pick'em or bad lines)
remaining_missing = games_df["underdog_team"].isna().sum()
print("Remaining missing underdog_team:", remaining_missing)

Remaining NaNs in spreads:
spread_home    0
spread_away    0
dtype: int64
Remaining missing underdog_team: 19


19 entries are still missing the `underdog_team` label, why could that be?

In [18]:
missing_ud = games_df[games_df["underdog_team"].isna()]
missing_ud

,game_id,season,season_type,week,game_date,home_team,away_team,final_home_score,final_away_score,score_diff,home_win,home_plays,home_pass_plays,home_rush_plays,home_yards_total,home_epa_total,home_epa_mean,home_success_rate,home_touchdowns,home_interceptions,home_fumbles_lost,home_explosive_plays,away_plays,away_pass_plays,away_rush_plays,away_yards_total,away_epa_total,away_epa_mean,away_success_rate,away_touchdowns,away_interceptions,away_fumbles_lost,away_explosive_plays,underdog_win,underdog_team,spread_home,spread_away
59,2005_06_CAR_DET,2005,REG,6,2005-10-16,DET,CAR,20,21,-1,0,84,39,23,209.0,-26.717056,-0.318060,0.333333,0,1,3,1,78,39,23,317.0,-23.347670,-0.299329,0.405063,5,3,1,2,0.0,None,0.0,0.0
92,2005_09_CAR_TB,2005,REG,9,2005-11-06,TB,CAR,14,34,-20,0,87,48,18,270.0,-14.033453,-0.161304,0.436782,3,2,2,2,70,20,31,287.0,10.843533,0.154908,0.450704,3,0,0,3,0.0,None,-0.0,0.0
99,2005_09_PHI_WAS,2005,REG,9,2005-11-06,WAS,PHI,17,10,7,1,76,35,26,293.0,4.633140,0.060962,0.428571,2,0,1,2,78,41,21,336.0,-0.161535,-0.002071,0.423077,1,1,0,4,0.0,None,0.0,0.0
144,2005_13_DEN_KC,2005,REG,13,2005-12-04,KC,DEN,31,27,4,1,84,28,37,421.0,13.278149,0.158073,0.571429,4,2,1,5,75,31,29,388.0,12.314595,0.164195,0.539474,3,2,0,3,0.0,None,-0.0,0.0
360,2006_13_JAX_MIA,2006,REG,13,2006-12-03,MIA,JAX,10,24,-14,0,81,47,19,309.0,-4.302337,-0.053115,0.456790,1,2,0,2,70,31,26,353.0,12.292267,0.175604,0.464789,3,0,1,5,0.0,None,0.0,0.0
363,2006_13_NYJ_GB,2006,REG,13,2006-12-03,GB,NYJ,10,38,-28,0,85,49,23,351.0,-9.894920,-0.116411,0.441860,1,2,1,2,82,38,31,441.0,22.518822,0.274620,0.597561,5,2,0,6,0.0,None,-0.0,0.0
554,2007_11_TEN_DEN,2007,REG,11,2007-11-19,DEN,TEN,34,20,14,1,65,24,24,359.0,18.156658,0.279333,0.469697,3,0,0,4,92,49,24,423.0,5.330612,0.057941,0.478261,3,2,2,2,0.0,None,0.0,0.0
571,2007_13_CLE_ARI,2007,REG,13,2007-12-02,ARI,CLE,27,21,6,1,82,30,31,302.0,4.021087,0.049038,0.421687,2,1,1,0,81,45,21,381.0,0.248802,0.003072,0.506173,3,2,1,4,0.0,None,0.0,0.0
1173,2010_10_HOU_JAX,2010,REG,10,2010-11-14,JAX,HOU,31,24,7,1,92,37,34,491.0,19.279027,0.209555,0.580645,4,0,0,4,75,37,19,395.0,13.225197,0.176336,0.466667,3,0,1,3,0.0,None,0.0,0.0
1222,2010_14_CLE_BUF,2010,REG,14,2010-12-12,BUF,CLE,13,6,7,1,83,27,40,323.0,-7.304579,-0.088007,0.470588,1,0,1,2,60,22,25,187.0,-13.844851,-0.230748,0.483333,0,1,2,2,0.0,None,0.0,0.0


Turns out these are games which had equal spread, so both teams were equally matched and had the same odds. Thus, nobody was really the underdog in this case. These matches are few enough to be treated as an exception, so we will drop them. 

In [19]:
games_df = games_df.dropna(subset=["underdog_team"]).reset_index(drop=True)
remaining_missing = games_df["underdog_team"].isna().sum()
print("Remaining missing underdog_team:", remaining_missing)

Remaining missing underdog_team: 0


This leaves us with:

In [20]:
games_df.shape

(2326, 37)

Let's save this dataset:

In [21]:
games_df.to_csv("final_data.csv")

In order to keep our work modular and testable, we decided end this notebook at data cleaning. For data exploration and visualization, you may move on to ***Notebook-3***, once you have `final_data.csv` saved and ready. 